# Overview

This notebook outlines a workflow and executes key code for batch processing of autotracking.



# Packages, paths, and parameters
You'll need to execute the next cell for all of the code that follows. Make sure that the root_code and root_proj paths are correct for your system.

In [1]:
import sys
import os
import def_definepaths as dd
import def_acquisition as da
import numpy as np
import pandas as pd
import cv2 as cv

# Root path for code on your system (can be set to None if you want to use give_paths to define them)
root_code = None

# Root path for the data and videos for the project
root_proj = None

# Get paths (specific to system running code)
path = dd.give_paths(root_code=root_code, root_proj=root_proj)

# Add path to kineKit 'sources' directory
sys.path.insert(0, path['kinekit'] + os.sep + 'sources')

# Import from kineKit
import acqfunctions as af
import videotools as vt

# Raw video extension
vid_ext_raw = 'MOV'

# Compressed video quality level (0 to 1, where 1 is no compression)
vid_quality = 0.75

# Create mask

You'll need to create a new mask whenever the position of the tank changes in the video frame. Follow the following steps to create a new mask.

## Measure the region-of-interest (ROI)

Here the aim is to define an ellipical region-of-interest where the tank resides within the video frames.

1. Set the 'make_video' column in the 'experiment_log.csv' file to a value of 1 for the video for which you want to create a mask.

1. Set all other 'make_video' rows to 0. If there is more than one row set to 1 then the code will use the first in the list.

1. Run the cell below to save a single frame from the video, which is saved to the 'masks' directory.

1. After the frame is generated, open it up in [Fiji](https://imagej.net/software/fiji/downloads) (or ImageJ), use the ellipse tool to draw around the tank margin. Be sure not to cut off any portion of the tank where a fish might end up. All areas outside of the ROI will be ignored.

1. Once the ellipse has been drawn, select Analyze:Measure in the pull-down menus to find the region-of-interest coordinates.

1. Enter the value for 'BX' as 'roi_x' in the experiment_log (enter values online in Google Sheets). Do the same for BY->roi_y, Width->roi_w, Height->roi_h. Copy and paste the values to all rows corresponding to videos that use that same ROI.

1. Make sure that the local version of experiment_log matches the values as the Google Sheet. This can be done by redownloading the CSV file, or copy and pasting values to the local copy. 



In [ ]:
# Index of video in cat list to extract video
vid_index = 0

# Extract experiment catalog info
cat = af.get_cat_info(path['cat'])

# Filename for frame of current sequence
filename = cat.date[vid_index] + '_' + format(cat.trial_num[vid_index],'03') + '_frame1'

# Define path for video
full_path = path['vidin'] + os.sep + cat.video_filename[vid_index] + '.' + vid_ext_raw

# Path for video frame
frame_path = path['mask'] + os.sep + filename + '.jpg'

# Extract frame and save to 'mask' directory
im = vt.get_frame(full_path)
result = cv.imwrite(frame_path, im)

if result is False:
    print('Save to the following path failed: ' + frame_path)
else:
    print('Video frame saved to: ' + frame_path)

## Create the mask image

1. As in step above, the cell below will use the first row for which make_video=1 in the 'experiment_log.csv' file to define the ROI for the mask, so adjust the spreadsheet accordingly.

1. The code will prompt you to choose a filename for the mask image and will save that file to the 'masks' directory.

1. Once completed, enter the mask filename (without the 'png' extension) into the mask_filename column of experiment_log for all experiments that should use that mask.

In [ ]:
# Index of video in cat list to extract video
vid_index = 0

# Extract experiment catalog info
cat = af.get_cat_info(path['cat'])

# Define path to raw video
full_path = path['vidin'] + os.sep + cat.video_filename[vid_index] + '.' + vid_ext_raw

# Extract video frame 
im = vt.get_frame(full_path)

# Make frame a gray field
im = int(256/2) + 0*im

# Extract roi coordinates
x_roi = float(cat.roi_x[vid_index])
y_roi = float(cat.roi_y[vid_index])
w_roi = float(cat.roi_w[vid_index])
h_roi = float(cat.roi_h[vid_index])
xC = x_roi + w_roi/2
yC = y_roi + h_roi/2
dims = (int(np.ceil(w_roi/2)), int(np.ceil(h_roi/2)))
cntr = (int(x_roi + w_roi/2), int(y_roi + h_roi/2))

# Define circular image for mask
im = cv.ellipse(im, cntr, dims, angle=0, startAngle=0, endAngle=360, color=(255,255,255),thickness=-1)

# Start transparent image as a bunch of opaque white pixels
trans_img = int(255)*np.ones((im.shape[0], im.shape[1], 4), dtype=np.uint8)

# Make pixels around circle opaque gray
trans_img[:,:,0] = int(256/2)
trans_img[:,:,1] = int(256/2)
trans_img[:,:,2] = int(256/2)

# Set opacity (4th channel in image) to zero at white circle pixels in im
trans_img[np.where(np.all(im[..., :3] == 255, -1))] = 0

# Filename for frame of current sequence
filename = cat.date[vid_index] + '_' + format(cat.trial_num[vid_index],'03') + '_mask'

# Output mask file
mask_path = path['mask'] + os.sep + filename + '.png'

# Write mask image to disk
result = cv.imwrite(mask_path, trans_img)
# result = cv.imwrite(mask_path, im)

if result is False:
    print('Saving mask image the following path failed: ' + mask_path)
else:
    print('Mask image saved to: ' + mask_path)


# Create compressed movies

The code below will generate compressed videos for all experiments in experiment_log where analyze=1 and make_video=1. 
This is done in three steps.
First, uncompressed masked movies are created and stored in the 'tmp' directory, then compressed and cropped movies are saved in 'pilot_compressed' (the tmp movies are then deleted), and finally there is cleanup step, where the final videos are verified and tmp videos are deleted.

In [ ]:
# The default is to not parallelize the code
para_mode = False

## Converting in batches

To speed up the conversion for batches of videos, we've parallelized the code (though you can skip this complexity by setting para_mode=False).
So, you will want to adjust the num_cores parameter below to the number of cores in your machine.
However, the code cannot handle a situations where you are converting more movies than there are cores.
So, if you are converting fewer movies than you have cores, then allocate the number of cores to the total number of movies.

In order for the parallel processing to work, open a terminal and activate the environment for this project (e.g., 'conda active wake' for the 'wake' environment), and run the following code, where the final number is the number of cores to be run:

> ipcluster start -n 8

You should get a message that "Engines appear to have started successfully", if things are working.

Next, execute the batch_command function below.

In [ ]:
# Whether to use parallel processing 
# (set to False when running this on a small number of movies)
para_mode = True

def batch_command(cmds):
    """ Runs a series of command-line instructions (in cmds dataframe) in parallel """

    import ipyparallel as ipp
    import subprocess

    # Set up clients 
    client = ipp.Client()
    type(client), client.ids

    # Direct view allows shared data (balanced_view is the alternative)
    direct_view = client[:]

    # Function to execute the code
    def run_command(idx):
        import os
        output = os.system(cmds_run.command[idx])
        # output = subprocess.run(cmds_run.command[idx], capture_output=True)
        # result = output.stdout.decode("utf-8")
        return output
        # return idx

    direct_view["cmds_run"] = cmds

    res = []
    for n in range(len(direct_view)):
        res.append(client[n].apply(run_command, n))
    
    return res

## Step 1 of 3
The compressed videos are created in three steps. 
First, uncompressed videos are generated with a mask and saved in the 'tmp' folder (within the video directory). 
The cell below accomplishes this step, but note that the work is accomplished by sending the job to the terminal, which makes it look like here like the job is complete. 
After you have started the job, you can track its progress below.

In [ ]:
# Extract experiment catalog info
cat = af.get_cat_info(path['cat'], include_mode='make_video')

# Make the masked videos (stored in 'tmp' directory)
cmds = af.convert_masked_videos(cat, in_path=path['vidin'], out_path=path['tmp'], 
            maskpath=path['mask'], vmode=False, imquality=1, para_mode=para_mode, 
            echo=False, out_name='date_trial')

if para_mode:
    # Run FFMPEG commands in parallel
    results = batch_command(cmds)

### For batches
For parallel mode, you can check on the status of this job with the print command in the cell below.

In [ ]:
if para_mode:
    print(results)

In response to executing the above cell, you will see the following result for each video that is being worked on:

> <AsyncResult(run_command): pending>

However, if there is a problem, then you will see something like this:

> <AsyncResult(run_command): failed>

When the job finishes correctly, then it will look like this:

> <AsyncResult(run_command): finished>

You can keep re-running the cell below each time you want to check on its status. If you have a lot of movies, then this can take a long time. 
Do not move to the next step until all videos are finished.

## Step 2 of 3
In the second step, the masked videos are compressed and cropped with parallel processing.
This is achieved in a similar way, with the following cell.

In [ ]:
# Thickness (in pixels) of border around ROI
border_pix = 10

# Formulate command(s) for conversion
cmds = af.convert_videos(cat, in_path=path['tmp'], out_path=path['vidout'], 
            in_name='date_trial', out_name='date_trial', vmode=False, imquality=vid_quality, 
            suffix_in='mp4', para_mode=para_mode, echo=False, border_pix=border_pix)

if para_mode:
    # Run FFMPEG commands in parallel
    results = batch_command(cmds)

### For batches
Again, you can check on the job status (parallel mode only):

In [ ]:
if para_mode:
    print(results)

## Step 3 of 3 
Once the job is finished, then you can survey the directories to make sure that all the videos have been compressed and the tmp files will be deleted:

In [ ]:
# Step through each experiment
for c_row in cat.index:

    # Define trial filename
    trialnum = str(int(cat.trial_num[c_row]))
    trialnum = '00' + trialnum[-3:]
    datetrial_name = cat.date[c_row] + '_' + trialnum

    # Temp video path
    vid_tmp_path = path['tmp'] + os.sep + datetrial_name + '.mp4'

    # Output video path
    vid_out_path = path['vidout'] + os.sep + datetrial_name + '.mp4'

    # Check whether output file was not made
    if not os.path.isfile(vid_out_path):

        print('   Output movie NOT created successfully: ' + vid_out_path)

        if os.path.isfile(vid_tmp_path):
            print('   Also, temp. movie NOT created successfully: ' + vid_tmp_path)
        else:
            print('   But, temp. movie created successfully: ' + vid_tmp_path)

    # If it was . . .
    else:
        print('   Output movie created successfully: ' + vid_out_path)

        # Delete temp file
        if os.path.isfile(vid_tmp_path):
            os.remove(vid_tmp_path)

# TGrabs and TRex

## Parameters

TGrabs parameters are described in the [TRex documentation](https://trex.run/docs/parameters_tgrabs.html)

In [2]:
# Parameter list for TGrabs 
# (These were listed in the spreadsheet)
param_list_tgrabs = [
    ['threshold', '9'], 
    ['use_adaptive_threshold', 'true'], 
    ['meta_real_width', '120'],
    ['dilation_size','2.5'],
    ['adaptive_threshold_scale','20'],
    ['blob_size_range','[0.01,0.4]']
    ]

# Specify list of parameter values for TRex, all listed as strings
param_list_trex = [
    ['gui_show_recognition_bounds','false'],
    ['gui_show_posture','false'],
    ['gui_show_paths','false'],
    ['gui_show_outline', 'false'], 
    ['gui_show_midline', 'false'], 
    ['gui_show_blobs', 'false'],
    ['gui_show_number_individuals', 'true'],
    ['gui_foi_name','manual match'],
    # ['heatmap_ids','[4]'],
    ['output_format','csv'],
    ['output_invalid_value','nan'],
    # ['gui_zoom_limit','[100,100]'],
    ['frame_rate','59']
    # ['blob_size_ranges','[0.01,0.4]']
    ]

# param_list_trex = [
#     ['track_threshold','9'],
#     ['track_max_speed','60'],
#     ['track_max_individuals','0'],
#     ['gui_show_recognition_bounds','False'],
#     ['gui_show_posture','false'],
#     ['gui_show_paths','false'],
#     ['gui_show_outline', 'false'], 
#     ['gui_show_midline', 'false'], 
#     ['gui_show_blobs', 'false'],
#     ['gui_show_number_individuals', 'true'],
#     ['gui_foi_name','manual match'],
#     ['heatmap_ids','[4]'],
#     ['output_invalid_value','nan'],
#     ['gui_zoom_limit','[100,100]'],
#     ['frame_rate','59'],
#     ['blob_size_ranges','[0.01,0.4]']
#     ]

# TGrabs generation of movies for TRex

## Execute TGrabs
Cell below generates dv videos, which will be used by TRex, from compressed videos.
This will be completed for each row of cat.

In [ ]:
# Define input parameter list as dataframe
# param_input = pd.DataFrame(param_list_tgrabs, columns=['param_name', 'param_val'])
param_input = pd.DataFrame([])

# Add the TRex parameter listing to the TGrabs parameters
# (might improve the preliminary tracking)
# param_input.append(param_list_trex)

# Extract experiment catalog info
cat = af.get_cat_info(path['cat'], include_mode='analyze')

# Loop thru each video listed in cat
for c_row in cat.index:

    # Define trial filename
    trialnum = str(int(cat.trial_num[c_row]))
    trialnum = '00' + trialnum[-3:]
    datetrial_name = cat.date[c_row] + '_' + trialnum

    # Define and check input path
    path_in = path['vidout'] + os.sep + datetrial_name + '.mp4'
    if not os.path.isfile(path_in):
        raise OSError('Video file does not exist: ' + path_in)

    # Output path
    path_out = path['vidpv'] + os.sep + datetrial_name + '.pv'

    # Path to save settings table
    path_settings = path['settings'] + os.sep + datetrial_name + '_tgrabs_settings.csv'

    # Start formulating the TGrabs command
    command = f'tgrabs -i {path_in} -o {path_out} '

    # Add additional command
    # command += '-averaging_method mode '

    # Loop thru each parameter value included in cat
    for idx in param_input.index:
        command += '-' + str(param_input.param_name[idx]) + ' ' + str(param_input.param_val[idx]) + ' '

    # Write settings to csv
    param_input.to_csv(path_settings, index=False)
    
    # Execute at the command line
    os.system(command)
    # print(command)

# Running TRex

## Batch execution of experiment videos with TRex

Be sure that experiment_log.csv includes the name of the settings file, in the settings directory, that TRex can use to 

In [5]:
# Create the pandas DataFrame that holds the parameter values
params_trex = pd.DataFrame(param_list_trex, 
                    columns=['param_name', 'param_val'])

# Extract experiment catalog info
cat = af.get_cat_info(path['cat'], include_mode='analyze')

# Loop thru each video listed in cat
for c_row in cat.index:

    # Define trial filename
    trialnum = str(int(cat.trial_num[c_row]))
    trialnum = '00' + trialnum[-3:]
    datetrial_name = cat.date[c_row] + '_' + trialnum

    # Define and check input path
    path_in = path['vidpv'] + os.sep + datetrial_name + '.pv'

    # Check for input file
    if not os.path.isfile(path_in):
        raise OSError('Video file does not exist: ' + path_in)
    
    # Where data will be saved
    data_path = path['data_raw'] 

    # Overwrite number of fish
    # params_trex.track_max_individuals = str(int(cat.fish_num[c_row]))

    # Settings path
    # path_settings = path['settings'] + os.sep + datetrial_name + '.settings'
    path_settings = datetrial_name + '.settings'

    # Start formulating the TGrabs command
    command = f'trex -i {path_in} -output_dir {data_path} -settings_file {path_settings} '

    # Add path for settings
    # command += f'-settings_file {path_settings} '

    # Get max number of fish from spreadsheet
    command += f'-track_max_individuals {str(int(cat.fish_num[c_row]))} '

    # Loop thru each parameter value included in cat
    for idx in params_trex.index:
        command += '-' + str(params_trex.param_name[idx]) + ' ' + str(params_trex.param_val[idx]) + ' '

    # Execute at the command line
    # result = os.system(command)
    print(command)

trex -i /Users/mmchenry/Documents/Projects/waketracking/video/pv/2022-10-03_001.pv -output_dir /Users/mmchenry/Documents/Projects/waketracking/data/raw -settings_file /Users/mmchenry/Documents/Projects/waketracking/data/settings/2022-10-03_001.settings -track_max_individuals 19 -gui_show_recognition_bounds false -gui_show_posture false -gui_show_paths false -gui_show_outline false -gui_show_midline false -gui_show_blobs false -gui_show_number_individuals true -gui_foi_name manual match -output_format csv -output_invalid_value nan -frame_rate 59 


## Export data in mat format

In [ ]:
from scipy.io import savemat
import glob

# Extract experiment catalog info 
cat = af.get_cat_info(path['cat'], include_mode=='analyze')

# Convert all npz files for an experiment to mat files.
da.raw_to_mat(cat)